<a href="https://colab.research.google.com/github/lsteffenel/CHPS0906/blob/main/TP1/StudentAdmissionsKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prédire les admissions des étudiants avec des réseaux neuronaux dans Keras
Dans ce bloc-notes, nous prédisons les admissions des étudiants aux études supérieures à l'UCLA (Université de Californie à Los Angeles) en fonction de trois éléments de données :
- Scores GRE (test)
- Scores GPA (notes)
- Classement (1-4)

L'ensemble de données provient à l'origine d'ici : http://www.ats.ucla.edu/

## Chargement des données
Pour charger les données et les formater correctement, nous utiliserons Pandas et Numpy.

In [ ]:
# Importing pandas and numpy
import pandas as pd
import numpy as np

# Reading the csv file into a pandas DataFrame
data = pd.read_csv('https://github.com/lsteffenel/CHPS0906/raw/refs/heads/main/TP1/data/student_data.csv')

# Printing out the first 10 rows of our data
data[:10]

## Visualisation des données

Commençons par créer une visualisation de nos données pour voir à quoi elles ressemblent. Pour obtenir un graphique en 2D, ignorons le rang.

In [ ]:
# Importing matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Function to help us plot
def plot_points(data):
    X = np.array(data[["gre","gpa"]])
    y = np.array(data["admit"])
    admitted = X[np.argwhere(y==1)]
    rejected = X[np.argwhere(y==0)]
    plt.scatter([s[0][0] for s in rejected], [s[0][1] for s in rejected], s = 25, color = 'red', edgecolor = 'k')
    plt.scatter([s[0][0] for s in admitted], [s[0][1] for s in admitted], s = 25, color = 'cyan', edgecolor = 'k')
    plt.xlabel('Test (GRE)')
    plt.ylabel('Grades (GPA)')

# Plotting the points
plot_points(data)
plt.show()

En gros, il semble que les élèves ayant obtenu les meilleures notes et les meilleurs tests aient réussi, tandis que ceux ayant obtenu les pires notes ont échoué, mais les données ne sont pas aussi bien séparées que nous l'espérions. Peut-être serait-il utile de prendre en compte le rang ? Créons 4 graphiques, chacun pour chaque rang.

In [ ]:
# Separating the ranks
data_rank1 = data[data["rank"]==1]
data_rank2 = data[data["rank"]==2]
data_rank3 = data[data["rank"]==3]
data_rank4 = data[data["rank"]==4]

# Plotting the graphs
plot_points(data_rank1)
plt.title("Rank 1")
plt.show()
plot_points(data_rank2)
plt.title("Rank 2")
plt.show()
plot_points(data_rank3)
plt.title("Rank 3")
plt.show()
plot_points(data_rank4)
plt.title("Rank 4")
plt.show()

Cela semble plus prometteur car le rang est bas, plus le taux d'acceptation est élevé (et oui, si vous pensez bien, rang 1 est plus petit que rang 4). Utilisons le rang comme l'une de nos entrées. Cependant, pour éviter des biais en fonction des valeurs numériques, nous devons encoder les rangs en format `one-hot encoding`.

## One-hot encoding du rang
Pour cela, nous utiliserons la fonction `get_dummies` dans pandas. Il est aussi possible de faire appel à scikit-learn, si vous voulez plus de flexibilité.

In [ ]:
# Make dummy variables for rank
one_hot_data = pd.concat([data, pd.get_dummies(data['rank'], prefix='rank')], axis=1)

# Drop the previous rank column
one_hot_data = one_hot_data.drop('rank', axis=1)

# Print the first 10 rows of our data
one_hot_data[:10]

## Mise à l'échelle des données
L'étape suivante consiste à mettre à l'échelle les données.
Nous remarquons que la plage des notes est de 1.0 à 4.0, tandis que la plage des scores aux tests est d'environ 200 à 800, ce qui est beaucoup plus large.

Cela rend leur gestion difficile pour un réseau neuronal car la différence d'ordre de grandeur peut jouer en faveur des scores test.

Afin de limiter cet écart, ajustons nos deux caractéristiques dans une plage de 0 à 1, en divisant les notes par 4.0 et le score au test par 800.

In [ ]:
# Copying our data
processed_data = one_hot_data[:]

# Scaling the columns
processed_data['gre'] = processed_data['gre']/800
processed_data['gpa'] = processed_data['gpa']/4.0
processed_data[:10]

## Répartition des données en un ensemble d'entraînement et un ensemble de tests

Afin de tester notre algorithme, nous allons repartir les données en un ensemble d'entraînement et un ensemble de tests. La taille de l'ensemble de tests sera de 10% des données totales.

In [ ]:
sample = np.random.choice(processed_data.index, size=int(len(processed_data)*0.9), replace=False)
train_data, test_data = processed_data.iloc[sample], processed_data.drop(sample)

print("Number of training samples is", len(train_data))
print("Number of testing samples is", len(test_data))
print(train_data[:10])
print(test_data[:10])

## Séparation des données en caractéristiques et cibles (étiquettes)

Maintenant, comme dernière étape avant l'entraînement, nous allons diviser les données en caractéristiques (X) et cibles (y).

De plus, dans Keras, nous devons indiquer que la sortie (`admit`) représente une catégorie et pas une valeur numérique. Nous le ferons avec la fonction `to_categorical`.

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Separate data and one-hot encode the output
# Note: We're also turning the data into numpy arrays, in order to train the model in Keras
features = np.array(train_data.drop('admit', axis=1)).astype('float32')
targets = np.array(keras.utils.to_categorical(train_data['admit'], 2))
features_test = np.array(test_data.drop('admit', axis=1)).astype('float32')
targets_test = np.array(keras.utils.to_categorical(test_data['admit'], 2))

print(features[:10])
print(targets[:10])

## Définition de l'architecture du modèle
C'est ici que nous utilisons Keras pour construire notre réseau de neurones.

In [ ]:
# Imports
import numpy as np
from keras import Layer
from keras import Optimizer

# Building the model
model = keras.Sequential()
model.add(keras.layers.Dense(128, activation='relu', input_shape=(6,)))
model.add(keras.layers.Dropout(.2))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(.1))
model.add(keras.layers.Dense(2, activation='softmax'))

# Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy','recall'])
model.summary()

Prenez 2 minutes et refléchissez à l'architecture proposée.
- quel est son type (dense, convolutionnelle) ?
- le nombre de neurones dans chaque couche vous semble adéquat ?
- comment feriez-vous pour décrire cette architecture en mode Functional (ici c'est en mode Sequential) ?



## Entraînement du modèle

In [ ]:
# Training the model
model.fit(features, targets, epochs=200, batch_size=100, verbose=0)

## Évaluation du modèle

In [ ]:
# Evaluating the model on the training and testing set
score = model.evaluate(features, targets)
print("\n Training Accuracy:", score[1])
score = model.evaluate(features_test, targets_test)
print("\n Testing Accuracy:", score[1])

In [ ]:
model.metrics_names

A votre avis, est-ce que ce modèle a une bonne accuracy ? Comment feriez-vous pour rajouter le `recall` à la liste de métriques à surveiller pendant l'entraînement ?

*vous pouvez modifier le code pour afficher l'accuracy et le recall*

## Défi : Jouez avec les paramètres !
Nous avons pris plusieurs décisions comme par exemple le nombre de couches, la taille des couches, le nombre d'époques, etc.

C'est à votre tour de jouer avec les paramètres ! Pouvez-vous améliorer la précision ?

Essayez aussi d'autrers paramètres :
- Fonction d'activation : relu et sigmoïde
- Fonction de perte : categorical_crossentropy, mean_squared_error
- Optimiseur : rmsprop, adam, ada